# Popular Hacker News stories analyzed with AlchemyAPI

[Hacker News](https://news.ycombinator.com/) is a social news website focusing on computer science and entrepreneurship. It is run by Paul Graham's investment fund and startup incubator, [Y Combinator](https://www.ycombinator.com/).

Hacker News is community based. Articles are posted and upvoted by users. The most popular articles make it to the top of the site. The algorithm that computes the top ranked articles is largely impacted by time, giving new articles a significant boost. After all, it's a news site.

Content that can be submitted to Hacker News is defined as "anything that gratifies one's intellectual curiosity". If you haven't seen the site yet, [check it out](https://news.ycombinator.com/). You can find more details about the Hacker News algorithm in [this blog post](http://amix.dk/blog/post/19574).

If you're already an avid reader of Hacker News you might have wondered **"Is there a topic commonality between the popular Hacker News stories?** or put differently if you're an author - **"What topic do I have to write about to make it to the top of Hacker News?**

This notebook is trying to answer these questions using the [Hacker News API](https://github.com/HackerNews/API) and the [AlchemyAPI](http://www.alchemyapi.com/) to get ranked concepts for textual data. Here are the steps we'll take:

1. [Query the most popular Hacker News stories](#Query-the-most-popular-Hacker-News-stories)
1. [Save the current 500 most popular Hacker News stories](#Save-the-current-500-most-popular-Hacker-News-stories)
1. [Tag concepts of Hacker News stories using AlchemyAPI](#Tag-concepts-of-Hacker-News-stories-using-AlchemyAPI)

To run this notebook, [register for a free AlchemyAPI account](http://www.alchemyapi.com/api/register.html). After you receive your API key, paste your API key in the cell below:

In [1]:
api_key = 'PASTE_ALCHEMY_API_KEY_HERE'

Import required Python libraries:

In [2]:
import requests
import os
import pandas as pd
from datetime import datetime

## Query the current 500 most popular Hacker News stories

The [Hacker News API](https://github.com/HackerNews/API) provides an end point `topstories` that returns the 500 most highly rated stories at this time.

### Define Hacker News API request url strings

In [3]:
hacker_news_api_base_url = 'https://hacker-news.firebaseio.com/v0/'
hacker_news_feature_url_item = 'item/'
hacker_news_feature_url_topstories = 'topstories'
hacker_news_api_parameters = '.json?print=pretty'

### Define Hacker News API helper functions

In [4]:
def get_story_for_id(story_id):
    ''' Queries the Hacker News API for story information about for the given story_id. '''
    story_request_url = hacker_news_api_base_url + hacker_news_feature_url_item + unicode(story_id) + hacker_news_api_parameters
    story = requests.get(story_request_url).json()
    return story

In [5]:
def get_story_details(story):
    ''' Filter relevant story information from the given Hacker News API story object. '''
    # remove descendants from story (e.g. comments), because we don't use them
    if 'kids' in story: del story['kids']
    # encode text field content as ascii (work around IPython defect https://github.com/ipython/ipython/issues/6799)
    if 'title' in story: story['title'] = story['title'].encode('ascii', 'ignore')
    if 'text' in story: story['text'] = story['text'].encode('ascii', 'ignore')
    if 'url' in story: story['url'] = story['url'].encode('ascii', 'ignore')
    return story

In [6]:
def get_all_story_details(story_ids):
    ''' Queries Hacker News API for relevant story information for given list of story_ids. '''
    all_story_details = []
    for story_id in story_ids:
        all_story_details.append(get_story_details(get_story_for_id(story_id)))
    return all_story_details

### Query stories

With these helper functions in place, let's query the Hacker News API for the current top 500 stories:

In [7]:
current_top_500_stories_url = hacker_news_api_base_url + hacker_news_feature_url_topstories + hacker_news_api_parameters
current_top_500_stories = requests.get(current_top_500_stories_url).json()

Take a look at what we got to make sure we have a list of story ids:

In [8]:
current_top_500_stories

[9380468,
 9381187,
 9381792,
 9380962,
 9382302,
 9380792,
 9381524,
 9382295,
 9382551,
 9380181,
 9379582,
 9380165,
 9381506,
 9379981,
 9380010,
 9380558,
 9380513,
 9381547,
 9380635,
 9379432,
 9379968,
 9380202,
 9380338,
 9378861,
 9379307,
 9380622,
 9378971,
 9379496,
 9379015,
 9381473,
 9381161,
 9381305,
 9379931,
 9379224,
 9379514,
 9378852,
 9378497,
 9382055,
 9382050,
 9378934,
 9379299,
 9379817,
 9379285,
 9378952,
 9381376,
 9381315,
 9382576,
 9382559,
 9376391,
 9378884,
 9380090,
 9381437,
 9378953,
 9382206,
 9378633,
 9381048,
 9380339,
 9381017,
 9374927,
 9378755,
 9375886,
 9378914,
 9381209,
 9377304,
 9381403,
 9378227,
 9375565,
 9381866,
 9378449,
 9381901,
 9376086,
 9377834,
 9377951,
 9382315,
 9377574,
 9379409,
 9378041,
 9380621,
 9382155,
 9381833,
 9378898,
 9374314,
 9373599,
 9381517,
 9377669,
 9382027,
 9381139,
 9376940,
 9381559,
 9374146,
 9380232,
 9382080,
 9376148,
 9303396,
 9381235,
 9381021,
 9373933,
 9381521,
 9371854,
 9379270,


## Save the current 500 most popular Hacker News stories

The top 500 stories provided by the Hacker News API are a snapshot that reflect the currently most popular stories. To enable an analysis of the most popular stories over time, it is helpful to work with a larger corpus of stories.  

### Define story persistence helper function

Let's save the story ids on disk for future usage with the Python [pickle](https://docs.python.org/2/library/pickle.html) library provided by Pandas convenience methods [read_pickle](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.io.pickle.read_pickle.html) and [to_pickle](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_pickle.html):

In [9]:
story_ids_file_name = 'hacker_news_story_ids.pickle'

def update_saved_story_ids(story_ids, story_ids_file_name):
    ''' Read story ids from disk, merge with given story_ids, and save back to disk. '''
    file_story_ids = []
    try:
        file_story_ids = pd.read_pickle(story_ids_file_name)
    except IOError as err:
        # file for story ids does not yet exist, move on
        pass
    merged_story_ids = set(file_story_ids).union(set(story_ids))
    pd.Series(list(merged_story_ids)).to_pickle(story_ids_file_name)
    return merged_story_ids

In [10]:
story_ids_up_until_today = update_saved_story_ids(current_top_500_stories, story_ids_file_name)

### Query story details

Query details about all stories (e.g. title, url of linked article, publish time, score, number of descendenants). Let's see for how many stories we're going query details:

In [11]:
len(story_ids_up_until_today)

1053

Now, query the details and show a sample of the first five stories (If you happen to hit JSON errors, try running the cell again, as these seem to happen intermittently):

In [14]:
all_story_details = get_all_story_details(list(story_ids_up_until_today))
# optionally, comment the first line and uncomment the two lines below to use a subset of stories to work with to reduce subsequent API requests against AlchemyAPI
# top_10_stories = list(story_ids_up_until_today)[0:10]
# all_story_details = get_all_story_details(top_10_stories)
stories_df = pd.DataFrame.from_dict(all_story_details)
stories_df.head(5)

,by,dead,deleted,descendants,id,parts,score,text,time,title,type,url
0,mparr4,NaN,NaN,0,9367553,NaN,5,,1428933057,Negotiating HTTP/2: ALPN and the TLS Handshake,story,http://matthewparrilla.com/post/negotiation-ht...
1,ivank,NaN,NaN,29,9363458,NaN,102,,1428849266,The Essence of Peopling,story,http://www.ribbonfarm.com/2015/04/08/the-essen...
2,tokai,NaN,NaN,0,9367556,NaN,10,,1428933126,The Word-Space Model (2006) [pdf],story,https://www.sics.se/~mange/TheWordSpaceModel.pdf
3,spikels,NaN,NaN,6,9377110,NaN,39,,1429043471,"SpaceX Rocket lands on droneship, but too hard...",story,https://mobile.twitter.com/elonmusk/status/588...
4,gits1225,NaN,NaN,6,9367564,NaN,19,,1428933203,Programming languages shape the way their user...,story,http://www.technologyreview.com/review/536356/...


## Tag concepts of Hacker News stories using AlchemyAPI

One of the features provided by AlchemyAPI is [Concept Tagging](http://www.alchemyapi.com/api/concept/urls.html). It allows extracting concepts from web-based content available at a given URL. We're going to apply concept tagging to the URLs from the Hacker News stories.

### Define AlchemyAPI request URL strings

In [15]:
alchemy_api_base_url = 'http://access.alchemyapi.com/calls/url/'
alchemy_api_parameters = '?apikey=' + api_key + '&outputMode=json&url='
alchemy_feature_url_concepts = "URLGetRankedConcepts"

### Define AlchemyAPI helper functions

Implement a function to query the AlchemyAPI for concepts for a given url:

In [16]:
def get_concepts_for_url(story_url, story_urls_and_concepts):
    ''' Query AlchemyAPI concept tagging for given url and add result to given story_urls_and_concepts dictionary. '''
    if story_url in story_urls_and_concepts:
        # attempt to get concepts for story url from disk 
        concepts = story_urls_and_concepts.get(story_url)
    else:
        # no concepts available on disk for story url, query AlchemyAPI for concepts and add save for future use
        request_url = alchemy_api_base_url + alchemy_feature_url_concepts + alchemy_api_parameters + story_url
        concepts = requests.get(request_url).json()
        story_urls_and_concepts[story_url] = concepts
    return concepts

Let's test the function by running it against a `test_url` pointing to an article from cnn.com:

In [17]:
story_urls_and_concepts = {}
test_url = 'http://www.cnn.com/2009/CRIME/01/13/missing.pilot/index.html'
get_concepts_for_url(test_url, story_urls_and_concepts)  

{u'concepts': [{u'dbpedia': u'http://dbpedia.org/resource/Marshal',
   u'freebase': u'http://rdf.freebase.com/ns/m.01mz37',
   u'relevance': u'0.979535',
   u'text': u'Marshal'},
  {u'dbpedia': u'http://dbpedia.org/resource/United_States_Marshals_Service',
   u'freebase': u'http://rdf.freebase.com/ns/m.0p6f_',
   u'relevance': u'0.904977',
   u'text': u'United States Marshals Service',
   u'website': u'http://www.usdoj.gov/marshals',
   u'yago': u'http://yago-knowledge.org/resource/United_States_Marshals_Service'},
  {u'dbpedia': u'http://dbpedia.org/resource/Suicide',
   u'freebase': u'http://rdf.freebase.com/ns/m.06z5s',
   u'opencyc': u'http://sw.opencyc.org/concept/Mx4rwQrsYZwpEbGdrcN5Y29ycA',
   u'relevance': u'0.902347',
   u'text': u'Suicide'},
  {u'dbpedia': u'http://dbpedia.org/resource/Sheriff',
   u'freebase': u'http://rdf.freebase.com/ns/m.0mb31',
   u'opencyc': u'http://sw.opencyc.org/concept/Mx4rRWTpvi9AQdicUZy0P3lKVQ',
   u'relevance': u'0.784423',
   u'text': u'Sheriff'

You should see a JSON document containing list of `concepts` extracted from the website at the given url. Each `concept` is identified by the `text` and is assigned a `relevance` which measures how confident AlchemyAPI is that the website is talking about this `concept`. Based on the identified concept, the JSON also contains links to publicly available knowledge bases [DBPedia](http://dbpedia.org/) and [Yago](http://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/yago/). Feel free to test AlchemyAPI concept tagging for articles that you're interested in by replacing the `test_url`.

The free price tier for AlchemyAPI allows 1000 queries per day. To reduce the number of AlchemyAPI query requests, we create a dictionary `story_urls_and_concepts` to store the story url with the list of detected concepts:

In [26]:
story_urls_and_concepts_file_name = 'story_urls_and_concepts.pickle'

try:
    story_urls_and_concepts = pd.read_pickle(story_urls_and_concepts_file_name)
except IOError as err:
    # file for story urls and concepts does not yet exist, move on
    story_urls_and_concepts = {}
    pass

Now that we can extract concepts for articles at a given url we need to extract the concepts for a Hacker News `story_id`. For each identified concept we need to keep track how often and from what story it was extracted. We need to aggregate popularity measures like score and number of descendants from the stories. 

The resulting data structure is a dictionary of dictionaries containing the following information:

```
{
'Programming language': {
  concept_dict {
    'occurs' : 11    # there are 10 occurrences of the concept 'Programming language' in our stories
    'score'  : 543   # aggregated score of all stories containing concept 'Progamming language' is 543
    'ids'    : [123,456] # story ids of all stories containing concept 'Programming language'
    'descendants' 94  # aggregated number of all descendants of all stories containing 'Progamming language'
    'links' : ['www.cnn.com/programming_language', ... ]  # links to all stories about 'Progamming language'
  }
}
}
        
```

The following function aggregates all this information about all concepts extracted from all stories:

In [27]:
def get_concepts_for_id(story_id, all_concepts_dicts, story_urls_and_concepts):
    ''' Extracts concepts for given story_id and aggregates story popularity information. '''
    print "Querying concepts for story " + unicode(story_id) + "..."
    request_url = hacker_news_api_base_url + hacker_news_feature_url_item + unicode(story_id) + hacker_news_api_parameters
    print(request_url)
    story = requests.get(request_url).json()
    # ignore "Ask HN" and job posts, only consider actual stories
    if story.get('type') == 'story':
        # make sure story has url that links to article
        if story.get('url') is not None:
            # extract concepts using AlchemyAPI
            concept_result = get_concepts_for_url(story.get('url'), story_urls_and_concepts)
            if concept_result['status'] == 'OK':
                concepts = concept_result.get('concepts')
                for concept in concepts:
                    # check, if we previously encountered the concept in another article
                    concept_dict = {}
                    concept_text = concept.get('text')
                    # ignore concepts with low score
                    if (float(concept.get('relevance')) > 0.6):
                        concept_dict['occurs'] = 1
                        concept_dict['relevance'] = concept.get('relevance')
                        concept_dict['ids'] = [story_id]
                        concept_dict['score'] = story.get('score')
                        concept_dict['descendants'] = story.get('descendants')
                        concept_dict['links'] = [story.get('url')]
                        if concept_text in all_concepts_dicts:
                            # merge additional concept info with already existing concept info
                            # add up the scores and number of descendants by concept
                            already_existing_concept = all_concepts_dicts.get(concept_text)
                            already_existing_concept['occurs'] = already_existing_concept['occurs'] + 1
                            already_existing_concept['score'] = already_existing_concept['score'] + story.get('score')
                            already_existing_concept['descendants'] = already_existing_concept['descendants'] + story.get('descendants')
                            already_existing_concept['links'] = already_existing_concept['links'] + concept_dict['links']
                            already_existing_concept['ids'] = already_existing_concept['ids'] + concept_dict['ids']
                        else:
                            all_concepts_dicts[concept_text] = concept_dict
    return all_concepts_dicts

Let's test the `get_concepts_for_id` helper function by providing it a valid Hacker News story id:

In [28]:
all_concepts_dicts = {}
test_story_id = 9226497
all_concepts_dicts = get_concepts_for_id(test_story_id, all_concepts_dicts, story_urls_and_concepts)
print all_concepts_dicts

Querying concepts for story 9226497...
https://hacker-news.firebaseio.com/v0/item/9226497.json?print=pretty
{u'Split Airport': {'links': [u'http://lit.vulf.de/spotify-so-little/'], 'descendants': 295, 'ids': [9226497], 'score': 613, 'relevance': u'0.896593', 'occurs': 1}, u'Pool': {'links': [u'http://lit.vulf.de/spotify-so-little/'], 'descendants': 295, 'ids': [9226497], 'score': 613, 'relevance': u'0.941916', 'occurs': 1}}


You should see a dictionary of concepts with links to stories, score and descendant information. Feel free to enter different `test_story_id`s.

Now that we know our `get_concepts_for_id` function works, let's query and aggregate the concepts for all Hacker News stories:

In [29]:
len(story_urls_and_concepts)

1708

In [30]:
len(stories_df)

1053

In [31]:
story_counter = 1
for story_id in story_ids_up_until_today:
# optionally, comment the line above and uncomment the line below to limit requests to 10 stories
# for story_id in top_10_stories:
    all_concepts_dicts = get_concepts_for_id(story_id, all_concepts_dicts, story_urls_and_concepts)
    print 'Done. ' + unicode(story_counter) + ' stories queried.'
    story_counter = story_counter + 1

Querying concepts for story 9367553...
https://hacker-news.firebaseio.com/v0/item/9367553.json?print=pretty
Done. 1 stories queried.
Querying concepts for story 9363458...
https://hacker-news.firebaseio.com/v0/item/9363458.json?print=pretty
Done. 2 stories queried.
Querying concepts for story 9367556...
https://hacker-news.firebaseio.com/v0/item/9367556.json?print=pretty
Done. 3 stories queried.
Querying concepts for story 9377110...
https://hacker-news.firebaseio.com/v0/item/9377110.json?print=pretty
Done. 4 stories queried.
Querying concepts for story 9367564...
https://hacker-news.firebaseio.com/v0/item/9367564.json?print=pretty
Done. 5 stories queried.
Querying concepts for story 9261073...
https://hacker-news.firebaseio.com/v0/item/9261073.json?print=pretty
Done. 6 stories queried.
Querying concepts for story 9342994...
https://hacker-news.firebaseio.com/v0/item/9342994.json?print=pretty
Done. 7 stories queried.
Querying concepts for story 9375764...
https://hacker-news.firebaseio

Save the `story_urls_and_concepts` dictionary to disk for future use. The dictionary got created and updated while iterating through all `story_id`s and is valueable at this point, because it contains the concepts from querying the AlchemyAPI, which supports a limited number of requests per day. Without saving the `story_urls_and_concepts` to disk, we would hit that 1000 requests per day limit after just a few days of collecting story ids.

In [33]:
import pickle

with open(story_urls_and_concepts_file_name, 'wb') as story_urls_and_concepts_file:
    pickle.dump(story_urls_and_concepts, story_urls_and_concepts_file)      

## Evaluate the result

### Most popular stories by score

Create a DataFrame for the extracted concepts for tabular presentation and sort the concepts by score, showing the most popular topics at the top.

In [34]:
all_concepts_df = pd.DataFrame.from_dict(all_concepts_dicts, orient='index')
all_concepts_sorted_by_score_df = all_concepts_df.sort(columns='score', ascending=False)
all_concepts_sorted_by_score_df

,links,descendants,ids,score,relevance,occurs
Apple Inc.,[http://roadlesstravelled.me/2015/04/06/why-st...,1186,"[9342994, 9367868, 9360732, 9361163, 9378434, ...",2551,0.93003,16
Operating system,[https://www.kickstarter.com/projects/13814379...,808,"[9375914, 9359722, 9363857, 9347669, 9380621, ...",2333,0.987744,25
Computer program,[https://atmospherejs.com/chipcastledotcom/jsp...,580,"[9367859, 9367913, 9367618, 9380338, 9380339, ...",2159,0.780098,22
Java,"[https://github.com/jackm321/RustNN, https://a...",588,"[9277680, 9367859, 9380190, 9368075, 9368137, ...",2097,0.819112,18
Internet,[http://www.chinafile.com/conversation/new-chi...,807,"[9367601, 9363565, 9359799, 9372431, 9360421, ...",1944,0.94787,32
Programming language,[http://www.technologyreview.com/review/536356...,585,"[9367564, 9261073, 9363496, 9380089, 9380165, ...",1902,0.987098,49
Steve Jobs,[http://roadlesstravelled.me/2015/04/06/why-st...,833,"[9342994, 9363714, 9360732, 9366382, 9362328, ...",1871,0.758757,7
Google,[http://www.rawstory.com/rs/2015/04/5-worst-th...,928,"[9363714, 9196008, 9380421, 9368418, 6312903, ...",1618,0.882041,26
E-mail,[http://matthewparrilla.com/post/negotiation-h...,402,"[9367553, 9381017, 9364783, 9374335, 9375525, ...",1583,0.628929,10
Threads,[http://blog.rust-lang.org/2015/04/10/Fearless...,466,"[9355382, 9271246]",1573,0.774183,2


### Most discussed stories by number of comments

Let's see which topics are most discussed and result in the highest number of comments (story descendants):

In [35]:
all_concepts_sorted_by_descendants_df = all_concepts_df.sort(columns='descendants', ascending=False)
all_concepts_sorted_by_descendants_df

,links,descendants,ids,score,relevance,occurs
Apple Inc.,[http://roadlesstravelled.me/2015/04/06/why-st...,1186,"[9342994, 9367868, 9360732, 9361163, 9378434, ...",2551,0.93003,16
Google,[http://www.rawstory.com/rs/2015/04/5-worst-th...,928,"[9363714, 9196008, 9380421, 9368418, 6312903, ...",1618,0.882041,26
Steve Jobs,[http://roadlesstravelled.me/2015/04/06/why-st...,833,"[9342994, 9363714, 9360732, 9366382, 9362328, ...",1871,0.758757,7
Operating system,[https://www.kickstarter.com/projects/13814379...,808,"[9375914, 9359722, 9363857, 9347669, 9380621, ...",2333,0.987744,25
Internet,[http://www.chinafile.com/conversation/new-chi...,807,"[9367601, 9363565, 9359799, 9372431, 9360421, ...",1944,0.94787,32
Cryptography,[http://www.washingtonpost.com/world/national%...,597,"[9360553, 9377080, 9365026, 9365054, 9365302, ...",1036,0.733362,13
Java,"[https://github.com/jackm321/RustNN, https://a...",588,"[9277680, 9367859, 9380190, 9368075, 9368137, ...",2097,0.819112,18
Programming language,[http://www.technologyreview.com/review/536356...,585,"[9367564, 9261073, 9363496, 9380089, 9380165, ...",1902,0.987098,49
English-language films,"[http://arxiv.org/abs/1504.02179, http://sethg...",583,"[9367580, 9372086, 9368426, 9364426, 9372918, ...",1398,0.742263,17
Computer program,[https://atmospherejs.com/chipcastledotcom/jsp...,580,"[9367859, 9367913, 9367618, 9380338, 9380339, ...",2159,0.780098,22


You can be the judge whether these are the topics you would have expected to be on top. As you run this notebook over time, more stories will be available. You can also run this `Hacker News and AlchemyAPI.ipynb` notebook recurringly every day by running the notebook `Hacker News Runner.ipynb`. This will aggregate data over time and allow for more detailed analysis. 

In this notebook we showed the usage of the Hacker News API and the AlchemyAPI. We used the AlchemyAPI concept tagging feature to extract topics from the Hacker News stories. Finally, we sorted aggregated popularity information about the stories for each concept and showed it in a tabular form sorted by different popularity measures.

The invocation of the AlchemyAPI was rather simple. A lot of the code to write intermediate result to disk is to work around the 1000 requests per day limitation.

Concept detection is only one feature of the AlchemyAPI. Check out more features in the [API documentation](http://www.alchemyapi.com/api).

<div class="alert" style="border: 1px solid #aaa; background: radial-gradient(ellipse at center, #ffffff 50%, #eee 100%);">
<div class="row">
    <div class="col-sm-1"><img src="https://knowledgeanyhow.org/static/images/favicon_32x32.png" style="margin-top: -6px"/></div>
    <div class="col-sm-11">This notebook was created using <a href="https://knowledgeanyhow.org">IBM Knowledge Anyhow Workbench</a>.  To learn more, visit us at <a href="https://knowledgeanyhow.org">https://knowledgeanyhow.org</a>.</div>
    </div>
</div>